## FMRI Data by DeepNeuralNetwork 

In [1]:
import tensorflow as tf
from scipy.io import loadmat
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import mean_squared_error

### File load
* Observation : 1440
* 1dimension data X variable : 74484
* 4 categories 

In [6]:
file_name = "C://Users//korea//Desktop//class_material//2017-2//Deeplearning//assignment//BRI641_fMRI_data_new_sets.mat"
fmri = loadmat(file_name)
# print(m)

In [2]:
def reset_graph():
    tf.reset_default_graph()

### Hyperparameter
* Hidden layers : 2
* Nodes : [300, 100]

In [39]:
#input
n_inputs = 74484 #input fmri data

#hidden layers
n_hidden1 = 300 #first hiddenlayer nodes
n_hidden2 = 100 #second hiddenlayer nodes
#ouput layers
n_outputs = 4 #classification 4

learning_rate = 0.01 #learningrate
momentum = 0.008     #momentum 
n_epochs = 20        #epoch 20
batch_size = 30      #batch size
dropout_rate = 0.8   #dropout rate


### Network structure

* Initializer : dense default initializer ( get_variables : uniform )
* Drop_out : adapt whole networks to 80% dropout rates.
* Cost function : sparse softmax cross entropy function
* Optimizer : Accelerated nesterov momentum optimizer(Learning rate : 0.01, momentum : 0.008) if use momentum optimizer, optimizer don't converged.
* Activation function : ReLU

In [40]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [41]:
training = tf.placeholder_with_default(False, shape=(), name='training')
X_drop = tf.layers.dropout(X, dropout_rate, training=training)
with tf.name_scope("dnn"): 
    hidden1 = tf.layers.dense(X_drop, n_hidden1, activation=tf.nn.relu,name="hidden1") #use default initializer in dense method
    hidden1_drop = tf.layers.dropout(hidden1, dropout_rate, training=training)         #use default initializer in dense method
    hidden2 = tf.layers.dense(hidden1_drop, n_hidden2, activation=tf.nn.relu,name="hidden2") #use default initializer in dense method
    hidden2_drop = tf.layers.dropout(hidden2, dropout_rate, training=training) #use default initializer in dense method
    logits = tf.layers.dense(hidden2_drop, n_outputs, name="outputs") #use default initializer in dense method
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits) #use sparse softmax cross entropy function (classification)
    loss = tf.reduce_mean(xentropy, name="loss") 
    
with tf.name_scope("train"):
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum,use_nesterov = True) #Accelrated nesterove Momentum optimizer
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

## Cross Validation(LOOCV) and Training

In [46]:
init = tf.global_variables_initializer()#initialize parameter variables
saver = tf.train.Saver()

#needed data extract 
X_data = fmri["data_1d_x"]
Y_data = fmri["data_y"]


#Cross validation LOOCV
groups = []
for i in range(1,13):
    group = [i] * 120
    for i in group:
        groups.append(i)
logo = LeaveOneGroupOut()
logo.get_n_splits(X_data, Y_data, groups)

12

In [49]:
#Training Part
looop = []
for train_index, test_index in logo.split(X_data, Y_data, groups):  
    X_train, X_test = X_data[train_index], X_data[test_index] #train, test X
    y_train, y_test = Y_data[train_index], Y_data[test_index] #train, test Y

    with tf.Session() as sess:
        init.run()
        accuracy_test = []
        for epoch in range(n_epochs):
            i = 0
            for batch in range(len(X_train) // batch_size):
                X_batch = X_train[i:i + 30]
                y_batch = y_train[i:i + 30]
                i += 30
                sess.run(training_op, feed_dict={X: X_batch, y: y_batch.reshape(-1)})
            acc_train = accuracy.eval(feed_dict={X: X_train, y: y_train.reshape(-1)})
            acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test.reshape(-1)})
            accuracy_test.append(acc_test)
            print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
        looop.append(max(accuracy_test))
        save_path = saver.save(sess, "./my_model_final.ckpt")
        print("End")
        print("*"*30)
print("="*70)
print("final result")
print(looop)
print("Test accuracy: ", np.mean(looop))

0 Train accuracy: 0.773485 Test accuracy: 0.716667
1 Train accuracy: 0.946212 Test accuracy: 0.891667
2 Train accuracy: 0.951515 Test accuracy: 0.875
3 Train accuracy: 0.946212 Test accuracy: 0.85
4 Train accuracy: 0.975 Test accuracy: 0.891667
5 Train accuracy: 0.991667 Test accuracy: 0.916667
6 Train accuracy: 0.993939 Test accuracy: 0.916667
7 Train accuracy: 0.998485 Test accuracy: 0.916667
8 Train accuracy: 1.0 Test accuracy: 0.916667
9 Train accuracy: 1.0 Test accuracy: 0.916667
10 Train accuracy: 1.0 Test accuracy: 0.916667
11 Train accuracy: 1.0 Test accuracy: 0.916667
12 Train accuracy: 1.0 Test accuracy: 0.916667
13 Train accuracy: 1.0 Test accuracy: 0.925
14 Train accuracy: 1.0 Test accuracy: 0.925
15 Train accuracy: 1.0 Test accuracy: 0.925
16 Train accuracy: 1.0 Test accuracy: 0.933333
17 Train accuracy: 1.0 Test accuracy: 0.933333
18 Train accuracy: 1.0 Test accuracy: 0.933333
19 Train accuracy: 1.0 Test accuracy: 0.933333
End
******************************
0 Train accura

As a result of using the DNN model, the results from ROOCV are as follows.
[0.93333334, 0.98333335, 1.0, 0.94166666, 0.85833335, 0.93333334, 0.96666664, 1.0, 0.93333334, 0.92500001, 0.91666669, 0.85833335]
The accuracy average of ROOCV is 0.9375. In case of using default momentum optimizer, The accuracy average of ROOCV is 0.6327. So, in this case, Accelerated nesterov momentum optimizer function is better than momentum optimizer.

## FMRI Data by ConvolutionalNeuralNetwork

In [1]:
import tensorflow as tf
from scipy.io import loadmat
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import mean_squared_error

## Reset graph  and show graph function

In [2]:
def reset_graph():
    tf.reset_default_graph()

from IPython.display import clear_output, Image, display, HTML
def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

## Hyperparameter

* input data : 1*74484 
* 2-convolution layers  ( 32, 64 filters)
* 1-Fullyconnected Layers (64 nodes)
* batch_size : 30 

In [3]:
#Input data
height = 1
width = 74484
# depth = 46
channels = 1
n_inputs = height * width

#First CNN layer
conv1_fmaps = 32
conv1_ksize = 10
conv1_stride = 5
conv1_pad = "SAME"

#Second CNN layer
conv2_fmaps = 64
conv2_ksize = 10
conv2_stride = 5
conv2_pad = "SAME"
conv2_dropout_rate = 0.5

pool3_fmaps = conv2_fmaps

#FullyConnected layer
n_fc1 = 64

#Output
n_outputs = 4

n_epochs = 20 #epochs 
batch_size = 30 #batch_size


## CNN structure

* Initializer : dense default initializer ( get_variables : uniform )
* Cost function : sparse softmax cross entropy function
* Optimizer : AdamOptimizer(Learning rate : default)
* Activation function : ReLU
* filtersize : 10x10

In [4]:
reset_graph()

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, name="X")
    X_reshaped = tf.reshape(X, shape=[-1,width,channels])
    y = tf.placeholder(tf.int64, name="y")

conv1 = tf.layers.conv1d(X_reshaped,
                     filters=conv1_fmaps,
                     kernel_size=conv1_ksize,
                     strides=conv1_stride,
                     padding=conv1_pad,
                     activation=tf.nn.relu,
                     name="conv1")


conv2 = tf.layers.conv1d(conv1,
                         filters=conv2_fmaps,
                         kernel_size=conv2_ksize,
                         strides=conv2_stride,
                         padding=conv2_pad,
                         activation=tf.nn.relu,
                         name="conv2")

with tf.name_scope("pool3"):
    pool3 = tf.layers.max_pooling1d(conv2,
                                    pool_size=2,
                                    strides=2,
                                    padding="VALID")
    pool3_flat = tf.reshape(pool3, shape=[-1, pool3_fmaps*1490])
    
with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(pool3_flat, n_fc1, activation=tf.nn.relu, name="fc1")

with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, n_outputs, name="output")
#     print(logits)
    Y_proba = tf.nn.softmax(logits, name="Y_proba")
#     print(Y_proba)
with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
#     print(xentropy)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()


Tensor("output/output/BiasAdd:0", shape=(?, 4), dtype=float32)
Tensor("output/Y_proba:0", shape=(?, 4), dtype=float32)
Tensor("train/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits:0", shape=(?,), dtype=float32)


## Data and Cross Validation(LOOCV)

In [7]:
X_data = fmri["data_1d_x"]
Y_data = fmri["data_y"]


## leave on out cross validation - group making
groups = []
for i in range(1,13):
    group = [i] * 120
    for i in group:
        groups.append(i)
logo = LeaveOneGroupOut()
logo.get_n_splits(X_data, Y_data, groups)

12

## Learning Model

In [9]:
loop = []
for train_index, test_index in logo.split(X_data, Y_data, groups):
    X_train, X_test = X_data[train_index], X_data[test_index] #train, test X
    y_train, y_test = Y_data[train_index], Y_data[test_index] #Train, test Y
    with tf.Session() as sess:
        init.run()
        accuracy_test = []
        for epoch in range(n_epochs):
            i = 0
            for batch in range(len(X_train) // batch_size):
                X_batch = X_train[i:i + 30]
#                 print(X_batch.shape)
                y_batch = y_train[i:i + 30]
#                 print(y_batch.shape)
                i += 30
                sess.run(training_op, feed_dict={X: X_batch, y: y_batch.reshape(-1)})
            acc_train = accuracy.eval(feed_dict={X: X_train, y: y_train.reshape(-1)}) #train accuracy
            acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test.reshape(-1)})    #test accuracy
            accuracy_test.append(acc_test)
            print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
        loop.append(max(accuracy_test))
        save_path = saver.save(sess, "./my_model_final.ckpt")
        print("End")
        print("*"*30)
print("="*70)
print("final result")
print(loop)
print("Test accuracy: ", np.mean(loop))

0 Train accuracy: 0.520455 Test accuracy: 0.391667
1 Train accuracy: 0.89697 Test accuracy: 0.816667
2 Train accuracy: 0.962121 Test accuracy: 0.891667
3 Train accuracy: 0.959091 Test accuracy: 0.9
4 Train accuracy: 0.991667 Test accuracy: 0.966667
5 Train accuracy: 0.996212 Test accuracy: 0.975
6 Train accuracy: 1.0 Test accuracy: 0.966667
7 Train accuracy: 1.0 Test accuracy: 0.958333
8 Train accuracy: 1.0 Test accuracy: 0.933333
9 Train accuracy: 1.0 Test accuracy: 0.933333
10 Train accuracy: 1.0 Test accuracy: 0.933333
11 Train accuracy: 1.0 Test accuracy: 0.933333
12 Train accuracy: 1.0 Test accuracy: 0.933333
13 Train accuracy: 1.0 Test accuracy: 0.933333
14 Train accuracy: 1.0 Test accuracy: 0.933333
15 Train accuracy: 1.0 Test accuracy: 0.933333
16 Train accuracy: 1.0 Test accuracy: 0.933333
17 Train accuracy: 1.0 Test accuracy: 0.933333
18 Train accuracy: 1.0 Test accuracy: 0.933333
19 Train accuracy: 1.0 Test accuracy: 0.925
End
******************************
0 Train accuracy:

As a result of using the CNN model(2convlayer), the results from ROOCV are as follows.
[0.97500002, 0.99166667, 1.0, 0.95833331, 0.90833336, 0.97500002, 1.0, 1.0, 0.97500002, 0.97500002, 0.94166666, 1.0]
The accuracy average of ROOCV is 0.975.

## Conclusion

After implementing both DNN and CNN models, I have found that CNN model has a better performance of 0.9750 accuracy compared to 0.9375 accuracy of DNN. Therefore, for this fMRI data, CNN seems to be better suited than DNN.